<a href="https://colab.research.google.com/github/heidingaway/datapeople/blob/main/opengov_gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ckanapi

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 8.4 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=65f99cc5ee63302e8d860a0b81369f38f4a5b5c3cbeb9cc8c04e9534b589801d
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
from ckanapi import RemoteCKAN
import pandas as pd
import os
import zipfile
import yaml
import re
import unicodedata

In [8]:
# Access resource data via a web API
rc = RemoteCKAN('https://open.canada.ca/data/en/')
APIID = "56eed7af-6a70-48d7-a402-4082ebe1e3b5"

# Initialize an empty list to store all records
all_records = []

# Set the initial offset and desired limit
offset = 0
limit = 100  # Or any desired number of records per request

while True:
    result = rc.action.datastore_search(
        resource_id=APIID,
        limit=limit,
        offset=offset,
    )

    # Add the retrieved records to the list
    all_records.extend(result['records'])

    # If the number of records returned is less than the limit,
    # it means we've reached the end
    if len(result['records']) < limit:
        break

    # Increment the offset for the next request
    offset += limit

# Create the DataFrame from all the retrieved records
df = pd.DataFrame(all_records)

print(df.count())

_id             3
code            3
hr_title_en     3
dmo_title_en    3
hr_title_fr     3
dmo_title_fr    3
english_def     3
french_def      3
dtype: int64


In [10]:
# Clean up 'id' column
df = df.rename(columns={'_id': 'id'})

# Add a new 'title' column using the content of another column
col1 = 'hr_title_en'
df['title'] = df[col1]

In [12]:
# Confirm changes
print(df)

   id code     hr_title_en       dmo_title_en  hr_title_fr  \
0   1    1             Man                Man        Homme   
1   2    2           Woman              Woman        Femme   
2   3    3  Another gender  Non-binary person  Autre genre   

           dmo_title_fr                                        english_def  \
0                 Homme  This category includes persons whose reported ...   
1                 Femme  This category includes persons whose reported ...   
2  Personne non-binaire  "This category includes persons whose reported...   

                                          french_def           title  
0  Cette catégorie comprend les personnes dont le...             Man  
1  Cette catégorie comprend les personnes dont le...           Woman  
2  "Cette catégorie comprend les personnes dont l...  Another gender  


In [11]:
# Save as csv
output = df
output.to_csv('output.csv', index=False)
print(f"DataFrame saved to output.csv")

DataFrame saved to output.csv


In [13]:
# Create a directory to store the Markdown files
output_dir = 'outputmd1'
idfinal = 'code'
decno = "407.02."
os.makedirs(output_dir, exist_ok=True)

# Iterate through each row of the DataFrame
for index, row in output.iterrows():
    # Create the file name (e.g., row_0.md, row_1.md, etc.)
    file_name = os.path.join(output_dir, decno + f"{row[idfinal]} {row['title']}.md")

    # Convert the row to a dictionary
    row_dict = row.to_dict()

    # Convert the dictionary to YAML format
    yaml_content = yaml.dump(row_dict, default_flow_style=False)

    # Write the YAML content to the file (with .md extension)
    with open(file_name, 'w') as f:
        f.write("---\n" + yaml_content + "\n---")

print(f"Files with YAML content (saved as .md) are in the '{output_dir}' directory.")

# Create a zip file containing the Markdown files
zip_file_name = 'outputmd.zip'
with zipfile.ZipFile(zip_file_name, 'w') as zipf:
    for root, _, files in os.walk(output_dir):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, output_dir))

print(f"Markdown files zipped to '{zip_file_name}'.")

Files with YAML content (saved as .md) are in the 'outputmd1' directory.
Markdown files zipped to 'outputmd.zip'.
